! pip install pandasql

In [1]:
import json
import warnings  
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('precision', 0)
warnings.filterwarnings('ignore')

In [21]:
df=pd.read_csv('BikerDatav2.csv')

In [22]:
df.head()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23298154,Local365,19775,2020-12-10 14:52:24 UTC,2552,3rd/West,2552,3rd/West,41
1,23321039,Pay-as-you-ride,332,2020-12-18 15:43:07 UTC,2552,3rd/West,2552,3rd/West,74
2,23326829,Local365,19171,2020-12-20 15:44:21 UTC,2501,5th/Bowie,2552,3rd/West,62
3,24786257,Local365,19646,2021-08-08 17:31:07 UTC,3390,6th/Brazos,2552,3rd/West,63
4,24743726,Local365,17438,2021-08-03 14:34:41 UTC,4047,8th/Lavaca,2552,3rd/West,6


In [29]:
df['start_time'] = pd.to_datetime(df['start_time'])
# Extract the day of the week from 'start_time' and put it in words
df['day_of_week'] = df['start_time'].dt.day_name()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5619 entries, 0 to 5618
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             5619 non-null   int64              
 1   subscriber_type     5616 non-null   object             
 2   bikeid              5619 non-null   object             
 3   start_time          5619 non-null   datetime64[ns, UTC]
 4   start_station_id    5619 non-null   int64              
 5   start_station_name  5619 non-null   object             
 6   end_station_id      5619 non-null   int64              
 7   end_station_name    5619 non-null   object             
 8   duration_minutes    5619 non-null   int64              
 9   day_of_week         5619 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(4), object(5)
memory usage: 439.1+ KB


# I will be using pandas SQL to run my queries.

### QUESTION 1: On which day of the week do we on average have the longest trip?

Answer: Sunday appears to be the day if the week with the average longest trip of 79

In [31]:
# Define the SQL query to calculate the average duration of trips for each day of the week
query_avg_duration = '''
SELECT day_of_week         , 
       AVG(duration_minutes) AS avg_duration
FROM df
GROUP BY day_of_week         
ORDER BY avg_duration DESC
'''

# Execute the query using pandasql
result_avg_duration = sqldf(query_avg_duration)

# Print the result
result_avg_duration.head(10)

,day_of_week,avg_duration
0,Sunday,79.103
1,Saturday,62.635
2,Tuesday,44.561
3,Thursday,43.900
4,Friday,43.375
5,Monday,40.999
6,Wednesday,31.980


### QUESTION 2: What month/year has the most bike trips and what is the count of the trips?

Answer: september 2020 with 530 trip count

In [20]:
# Define the SQL query to count the number of trips for each month/year
query_most_trips = '''
SELECT strftime('%Y-%m', start_time) AS month_year, 
       COUNT(*) AS trip_count
FROM df
GROUP BY strftime('%Y-%m', start_time)
ORDER BY trip_count DESC
'''

# Execute the query using pandasql
result_most_trips = sqldf(query_most_trips)

# Print the result
result_most_trips.head(100)


,month_year,trip_count
0,2020-09,530
1,2021-03,507
2,2021-04,475
3,2020-11,448
4,2021-05,434
5,2020-10,427
6,2020-08,392
7,2021-06,350
8,2020-05,338
9,2020-07,323


### QUESTION 3: In the same table, return which particular trip has longest duration and the trip that has the shortest duration (return all the information(columns) on the table for this record)
If more than 1 record has the same duration, return the earliest trip [start time]
NB: Exclude 'Missing' and 'Stolen' as values in the end_station_name column.
Exclude trips that start and end at the same station.


In [10]:
# Define the SQL query to find the trips with the longest and shortest duration
query_longest_shortest_trip = '''
SELECT *
FROM df
WHERE duration_minutes = (SELECT MAX(duration_minutes)
FROM df WHERE end_station_name NOT IN ('Missing', 'Stolen') AND start_station_id != end_station_id)

OR

duration_minutes = (SELECT MIN(duration_minutes) 
FROM df WHERE end_station_name NOT IN ('Missing', 'Stolen') AND start_station_id != end_station_id)
ORDER BY start_time
LIMIT 2
'''

# Execute the query using pandasql
result_longest_shortest_trip = sqldf(query_longest_shortest_trip)

# Print the result
result_longest_shortest_trip.head(10)


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,21473408,Pay-as-you-ride,460,2020-01-15 09:14:08 UTC,2822,East 6th/Robert T. Martinez,2544,East 6th/Pedernales,2
1,21521905,Local365,435,2020-01-29 08:54:49 UTC,4055,11th/Salina,4055,11th/Salina,2
